# PME3481 - Controle e Aplicações

## Controlabilidade e Observabilidade

Supondo um SLIT, o código permite avaliar se é ou não controlável a partir das matrizes A e B, avaliando o gramiano, tal qual utilizando as matrizes A e C, avaliando a observabilidade.

In [10]:
import numpy as np
import control

As duas funções que realizam a análise de controlabilidade e observabilidade do sistema

In [11]:
def is_controllable(A, B):
    """
    Check if a system defined by its state matrix A and input matrix B is controllable.
    """
    return np.linalg.matrix_rank(control.ctrb(A, B)) == A.shape[0], control.ctrb(A, B)

In [12]:
def is_observable(A, C):
    """
    Check if a system defined by its state matrix A and output matrix C is observable.
    """
    return np.linalg.matrix_rank(control.obsv(A, C)) == A.shape[0], control.obsv(A, C)

Abaixo, as matrizes do sistema definidas

In [13]:
Xcm = 1.052
Xpe = 0.261
L = 3.304
Df = 0.127
Tlin = 230.57
Izz = 15.07
M = 23.545
g = 9.8055
dar = 1.091
Cd = 0.432
Vxlin = -10.0
Vvlin = 2.2

In [14]:
# X = [theta,xp,yp,thetap]
# Xp = [thetap,xpp,ypp,thetapp]
A = np.array([
    [0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
    [4.40301920e-01, -2.53575404e-03,  0.00000000e+00,  0.00000000e+00],
    [-9.79343488e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
    [4.91991402e-03,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00]
])

# u = [T,tau]

# B = np.array([[0, 0, 0],
#               [1/M, 0, 0],
#               [0, -Tlin/M, - (1/2)*dar*Cd*L*Df*2*Vvlin],
#               [0, -Tlin*Xcm/Izz, - (1/2)*dar*Cd*Xpe*L*Df*2*Vvlin]]) 

E = np.array([[0],
              [0],
              [ (- (1/2)*dar*Cd*L*Df*2*Vvlin)/(M*Izz)],
              [(- (1/2)*dar*Cd*Xpe*L*Df*2*Vvlin)/(M*Izz)]])

B = np.array([[0, 0],
              [1/M, 0],
              [0, -Tlin/M],
              [0, -Tlin*Xcm/Izz]])

C = np.array([
    [0.,  1.,  0.,  0.],
    [0.,  0.,  1.,  0.]
])

D = np.array([
    [0.,  0.],
    [0.,  0.]
])

cont = control.StateSpace(A, B, C, D)

sys = control.ss(A, B, C, D)
sys

<LinearIOSystem:sys[6]:['u[0]', 'u[1]']->['y[0]', 'y[1]']>

In [21]:
# Calculando a matriz de Controlabilidade
controllability_matrix = B
for i in range(1, 4):
    controllability_matrix = np.hstack((controllability_matrix, np.linalg.matrix_power(A, i) @ B))

# Calculando a matriz de Observabilidade
observability_matrix = C
for i in range(1, 4):
    observability_matrix = np.vstack((observability_matrix, C @ np.linalg.matrix_power(A, i)))

controllability_matrix, observability_matrix

(array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         -1.60955302e+01,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00, -7.91886247e-02],
        [ 4.24718624e-02,  0.00000000e+00, -1.07698197e-04,
          0.00000000e+00,  2.73096137e-07, -7.08689285e+00,
         -6.92504633e-10,  1.79706172e-02],
        [ 0.00000000e+00, -9.79273731e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  1.57630527e+02,
          0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00, -1.60955302e+01,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00, -7.91886247e-02,
          0.00000000e+00,  0.00000000e+00]]),
 array([[ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
          0.00000000e+00],
        [ 4.40301920e-01, -2.53575404e-03,  0.00000000e+00,
          0.00000000e+00],
        [-9.79343488e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e